In [3]:
import sys
if '..' not in sys.path:
    sys.path.append('..')
    
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import networkx as nx
import copy
import scipy as sp
import math
import seaborn
import pickle
import warnings
import os

from lib.mobilitysim import MobilitySimulator
from lib.town_data import generate_population, generate_sites, compute_distances
from lib.town_maps import MapIllustrator

# from lib.settings.town_settings_kaiserslautern import *
# from lib.settings.town_settings_ruedesheim import *
# from lib.settings.town_settings_tirschenreuth import *
# from lib.settings.town_settings_tubingen import *
from lib.settings.town_settings_sanfrancisco import *

# from lib.settings.town_settings_lausanne import *
# from lib.settings.town_settings_locarno import *
# from lib.settings.town_settings_lucerne import *
# from lib.settings.town_settings_jura import *

# Downsampling factor of population and sites
downsample = 100

# Country for different age groups
country = 'US' # 'GER', 'CH', 'US'

# Set the population generation mode.
# 3 options available: custom | random | heuristic
population_by = 'custom'

# Downsample population 
population_per_age_group = np.round(
    population_per_age_group * (town_population / (downsample * region_population))).astype('int').tolist()

print(f'Population per age group: {population_per_age_group}')

# change essential population ratio here
essential_to_total_ratio_list = [0.1]
summaries_ = []
summeries_sf = dict()
for essential_to_total_ratio in essential_to_total_ratio_list:
    num_essential_workers = np.floor(sum(population_per_age_group)*essential_to_total_ratio).astype('int').tolist()
    essential_type = 1 # 0:edu, 1:social, 2:office, 3:supermarket
    
    essential_distribution = np.array([
    0,      # 0-4
    0,      # 5-14
    0.04,   # 15-19
    0.06,   # 20-24
    0.55,   # 25-44
    0.30,   # 45-59
    0.05,   # 60-79
    0])     # 80+

    num_essential_per_age_group = np.floor(num_essential_workers * essential_distribution).astype('int').tolist()
    essential_prop_per_age_group = np.divide((num_essential_per_age_group),(population_per_age_group))
    print('essential per age group',num_essential_per_age_group,'total',np.sum(num_essential_per_age_group))

    # This block sends queries to OpenStreetMap
    # Make sure you have a working internet connection
    # If an error occurs during execution, try executing again 
    # If the call times out or doesn't finish, try restarting your internet connection by e.g. restarting your computer
    site_files=[]
    for root,dirs,files in os.walk(sites_path):
        for f in files:
            if f.endswith(".txt") and f != 'buildings.txt':
                site_files.append(sites_path+f)

    site_loc, site_type, site_dict, density_site_loc = generate_sites(bbox=bbox, query_files=site_files,
                                    site_based_density_file=sites_path+'buildings.txt')

    if downsample > 1:
        np.random.seed(42)
        # downsample sites like populatoin
        idx = np.random.choice(len(site_loc), size=int(len(site_loc) / downsample), 
                               replace=False, p=np.ones(len(site_loc)) / len(site_loc))
    
        site_loc, site_type = np.array(site_loc)[idx].tolist(), np.array(site_type)[idx].tolist()


    if region_population == town_population:
        tile_level = 15
    else:
        tile_level = 16

    if population_by == 'custom':
        # generate population across tiles based on density input
        print('Tile level: ', tile_level)
        home_loc, people_age, home_tile, tile_loc, people_household, essential_workers, num_essential_workers, essential_work_site = generate_population(
            density_file=population_path, bbox=bbox,
            population_per_age_group=population_per_age_group, 
            household_info=household_info, tile_level=tile_level, seed=42,
            essential_prop_per_age_group=essential_prop_per_age_group,
            site_type = site_type, essential_type = essential_type)
    
    elif population_by == 'random':
        # generate population across tiles uniformly at random
        home_loc, people_age, home_tile, tile_loc, people_household , essential_workers, num_essential_workers, essential_work_site = generate_population(
            bbox=bbox, population_per_age_group=population_per_age_group,
            tile_level=16, seed=42,
            essential_prop_per_age_group=essential_prop_per_age_group)
    
    elif population_by == 'heuristic':
        # generate population across tiles proportional to buildings per tile
        home_loc, people_age, home_tile, tile_loc, people_household , essential_workers, num_essential_workers, essential_work_site = generate_population(
            bbox=bbox, density_site_loc=density_site_loc,
            population_per_age_group=population_per_age_group, tile_level=16, seed=42,
            essential_prop_per_age_group=essential_prop_per_age_group)
         
    essential_to_total_pop_ratio = num_essential_workers/sum(population_per_age_group)
    tile_site_dist = compute_distances(site_loc, tile_loc)


    if country == 'US':
        mob_rate_per_age_per_type = [
           [5,    0,    0,   0   ], # 0-5
           [5,    0,    0,   0   ], # 5-14
           [5,    0,    3.6, 0.22], # 15-19
           [1.48, 3.52, 3.6, 0.21], # 20-24
           [0,    5,    3.6, 0.27], # 25-44
           [0,    5,    3.6, 0.36], # 45-59
           [0,    0,    3.6, 0.35], # 60-79
           [0,    0,    3.6, 0.35]] # 80+
    else:
        print('we only have US data at this point...')
    
    # edu, social, office, supermarket
    dur_mean_per_type = [5.0, 5.0, 0.64, 0.4]
    variety_per_type = [1, 1, 10, 2]
    
    mob_rate_per_age_per_type = np.divide(np.array(mob_rate_per_age_per_type), (24.0 * 7))

    if essential_type == 0:
        essential_mob_rate_per_type = [5, 0, 3.6, 0.27]
        essential_dur_mean_per_type = [5, 0, 0.64, 0.4]
    elif essential_type == 1:
        essential_mob_rate_per_type = [0, 5, 3.6, 0.27]
        essential_dur_mean_per_type = [0, 5, 0.64, 0.4]
    elif essential_type == 2:
        essential_mob_rate_per_type = [0, 0, 5, 0.27]
        essential_dur_mean_per_type = [0, 0, 5, 0.4]
    elif essential_type == 3:
        essential_mob_rate_per_type = [0, 0, 3.6,  5]
        essential_dur_mean_per_type = [0, 0, 0.64, 5]
    
    essential_mob_rate_per_type = np.divide(np.array(essential_mob_rate_per_type), (24.0 * 7))

    # time horizon
    max_time = 17 * 24.0 # data availability
    delta  = 4.6438 # as set by distributions

    kwargs = dict(home_loc=home_loc, people_age=people_age,
        site_loc=site_loc, site_type=site_type, site_dict=site_dict,
        num_people_unscaled=town_population,
        region_population=region_population,
        downsample = downsample,
        mob_rate_per_age_per_type=mob_rate_per_age_per_type,
        dur_mean_per_type=dur_mean_per_type,
        variety_per_type=variety_per_type,
        daily_tests_unscaled=daily_tests_unscaled,
        delta=delta,
        home_tile=home_tile,
        tile_site_dist=tile_site_dist,
        people_household = people_household,
        essential_workers=essential_workers,
        essential_mob_rate_per_type=essential_mob_rate_per_type,
        essential_dur_mean_per_type = essential_dur_mean_per_type,
        essential_work_site = essential_work_site,
        essential_type=essential_type) # emma

    with open(f'sf_type{essential_type}_prop{essential_to_total_ratio}_ds{downsample}.pk', 'wb') as fp:
        pickle.dump(kwargs, fp)


    mob_settings = 'sf_type'+str(essential_type)+'_prop'+str(essential_to_total_ratio)+'_ds'+str(downsample)+'.pk'  

    with open(mob_settings, 'rb') as fp:
        obj = pickle.load(fp)
    mob = MobilitySimulator(**obj)
    
    num_essen = 0
    num_non_essen = 0
    work_site = []
    for i in range(mob.num_people):
        if mob.essential_workers[i]:
            num_essen += 1
            work_site_temp = mob.essential_work_site[i]
            if work_site_temp not in work_site:
                if work_site_temp > -1 :
                    work_site.append(work_site_temp)
            #print('essential worker no:',i,' working at site:',mob.essential_work_site[i])
        else:
            num_non_essen += 1

    num_workers_at_work_site = [0]*len(work_site)
    for i in range(mob.num_people):
        for j in range(len(work_site)):
            if mob.essential_work_site[i] == work_site[j]:
                num_workers_at_work_site[j] += 1
    
    print('total population:',mob.num_people,', sites:', len(site_type))
    print('essential:',num_essen,', non_essential:',num_non_essen,', propotion:',num_essen/mob.num_people)
    print('essential proortion per age group:',essential_prop_per_age_group)
    print('Population (by Age): ', population_per_age_group,np.array(population_per_age_group).sum())
    print('Sites by type: ',  [(np.array(site_type) == i).sum() for i in range(len(dur_mean_per_type))])
    print('essential type:',essential_type)
    print('work_site:',work_site,len(work_site))
    print('number of workers at work site:',num_workers_at_work_site, np.array(num_workers_at_work_site).sum())

Population per age group: [387, 592, 308, 526, 3293, 1671, 1365, 362]
essential per age group [0, 0, 34, 51, 467, 255, 42, 0] total 849
Query 1 OK.
Query 2 OK.
Query 3 OK.
Query 4 OK.
Query 5 OK.
Tile level:  15
total population: 8499 , sites: 75
essential: 859 , non_essential: 7640 , propotion: 0.10107071420167078
essential proortion per age group: [0.         0.         0.11038961 0.09695817 0.14181597 0.15260323
 0.03076923 0.        ]
Population (by Age):  [387, 592, 308, 526, 3293, 1671, 1365, 362] 8504
Sites by type:  [5, 37, 28, 5]
essential type: 1
work_site: [12, 43, 35, 13, 67, 38, 20, 70, 0, 26, 62, 17, 60, 59, 48, 5, 45, 30, 58, 36, 21, 31, 25, 65, 4, 23, 56, 46, 39, 22, 40, 14, 63, 16, 55, 18, 15] 37
number of workers at work site: [19, 26, 27, 28, 31, 22, 20, 22, 16, 26, 25, 22, 28, 26, 27, 16, 18, 25, 20, 19, 35, 21, 28, 24, 23, 22, 31, 20, 22, 18, 19, 30, 20, 28, 18, 19, 18] 859
